In [1]:
import pandas as pd

import surprise
from surprise.prediction_algorithms import *
import pandas as pd
import numpy as np
import datetime as dt

from surprise import Dataset
from surprise import Reader

from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from surprise.model_selection import cross_validate

In [2]:
movies = pd.read_csv('./ml-latest-small/movies.csv')

In [3]:
link = pd.read_csv('./ml-latest-small/links.csv')

In [4]:
rating = pd.read_csv('./ml-latest-small/ratings.csv')

In [5]:
tags = pd.read_csv('./ml-latest-small/tags.csv')

In [6]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [7]:
movies.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [8]:
movies.title.value_counts()

Eros (2004)                                       2
War of the Worlds (2005)                          2
Emma (1996)                                       2
Saturn 3 (1980)                                   2
Confessions of a Dangerous Mind (2002)            2
                                                 ..
History of the World: Part I (1981)               1
Anchors Aweigh (1945)                             1
Midnight in the Garden of Good and Evil (1997)    1
Tsotsi (2005)                                     1
Last Man Standing (1996)                          1
Name: title, Length: 9737, dtype: int64

In [9]:
#Check to see if there are any duplicate titles
movies.title.duplicated().sum()

5

In [10]:
#Drop the 5 duplicated movie titles
movies.drop_duplicates(subset='title', inplace=True)

In [11]:
#Sanity check to ensure all duplicates were dropped from title column

movies.title.duplicated().sum()

0

In [12]:
movies[movies.title.duplicated() == True]

,movieId,title,genres


In [13]:
#Check length of DataFrame

len(movies)

9737

In [14]:
# Split title column into two new columns: Title and year 

movies['Title'] = movies['title'].str.split('(', n=1, expand=True)[0]

movies['year_released'] = movies['title'].str.split('(', n=1, expand=True)[1]

In [15]:
# Drop original column title

movies.drop(columns='title', inplace=True)

In [16]:
# Sanity Check 
movies

,movieId,genres,Title,year_released
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995)
1,2,Adventure|Children|Fantasy,Jumanji,1995)
2,3,Comedy|Romance,Grumpier Old Men,1995)
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995)
4,5,Comedy,Father of the Bride Part II,1995)
...,...,...,...,...
9737,193581,Action|Animation|Comedy|Fantasy,Black Butler: Book of the Atlantic,2017)
9738,193583,Animation|Comedy|Fantasy,No Game No Life: Zero,2017)
9739,193585,Drama,Flint,2017)
9740,193587,Action|Animation,Bungo Stray Dogs: Dead Apple,2018)


In [17]:
# Format year_released column

movies['year_released'] = movies.year_released.str.replace(')', '')

In [18]:
# Sanity Check to ensure formatting was completed
movies

,movieId,genres,Title,year_released
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Adventure|Children|Fantasy,Jumanji,1995
2,3,Comedy|Romance,Grumpier Old Men,1995
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Comedy,Father of the Bride Part II,1995
...,...,...,...,...
9737,193581,Action|Animation|Comedy|Fantasy,Black Butler: Book of the Atlantic,2017
9738,193583,Animation|Comedy|Fantasy,No Game No Life: Zero,2017
9739,193585,Drama,Flint,2017
9740,193587,Action|Animation,Bungo Stray Dogs: Dead Apple,2018


In [19]:
link

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [20]:
rating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [21]:
#converted timestamp to datetime
rating['timestamp'] = pd.to_datetime(rating['timestamp'], unit='s')

In [22]:
rating.rating.value_counts(normalize=True)

4.0    0.265957
3.0    0.198808
5.0    0.131015
3.5    0.130271
4.5    0.084801
2.0    0.074884
2.5    0.055040
1.0    0.027877
1.5    0.017762
0.5    0.013586
Name: rating, dtype: float64

In [23]:
rating.duplicated().sum()

0

In [24]:
rating.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [25]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [26]:
movie_rating = movies.join(rating, on='movieId', lsuffix='movie_', rsuffix='rating_', how='outer')

In [27]:
movie_rating

,movieId,movieIdmovie_,genres,Title,year_released,userId,movieIdrating_,rating,timestamp
0.0,1,1.0,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,1.0,3.0,4.0,2000-07-30 18:20:47
1.0,2,2.0,Adventure|Children|Fantasy,Jumanji,1995,1.0,6.0,4.0,2000-07-30 18:37:04
2.0,3,3.0,Comedy|Romance,Grumpier Old Men,1995,1.0,47.0,5.0,2000-07-30 19:03:35
3.0,4,4.0,Comedy|Drama|Romance,Waiting to Exhale,1995,1.0,50.0,5.0,2000-07-30 18:48:51
4.0,5,5.0,Comedy,Father of the Bride Part II,1995,1.0,70.0,3.0,2000-07-30 18:40:00
...,...,...,...,...,...,...,...,...,...
NaN,100831,NaN,NaN,NaN,NaN,610.0,166534.0,4.0,2017-05-03 21:53:22
NaN,100832,NaN,NaN,NaN,NaN,610.0,168248.0,5.0,2017-05-03 22:21:31
NaN,100833,NaN,NaN,NaN,NaN,610.0,168250.0,5.0,2017-05-08 19:50:47
NaN,100834,NaN,NaN,NaN,NaN,610.0,168252.0,5.0,2017-05-03 21:19:12


In [28]:
movie_rating.dropna(inplace=True)

In [29]:
movie_rating['userId'].nunique()

397

In [30]:
stats = movie_rating[['rating', 'timestamp']].describe()
stats

,rating
count,8110.000000
mean,3.461467
std,1.094116
min,0.500000
25%,3.000000
50%,3.500000
75%,4.000000
max,5.000000


## Split

In [31]:
train, test = train_test_split(movie_rating)

In [32]:
reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)

## Tuning

In [33]:
param_grid = {'k':[10, 50, 100],'min_k': [1, 5, 10]}
base_model = GridSearchCV(KNNBasic,param_grid=param_grid,joblib_verbose=5)
base_model.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    0.4s finished


In [34]:
base_model.best_params

{'rmse': {'k': 10, 'min_k': 1}, 'mae': {'k': 10, 'min_k': 1}}

In [ ]:
base_model.best_estimator

In [35]:
# param_grid = {'n_factors':[20, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
#               'reg_all': [0.4, 0.6]}
# gs_model = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
# gs_model.fit(jokes)

## Cross Validation

In [36]:
#instantiate KNN model 
model1=KNNBasic()

In [37]:
#instaniatite and perform cross val with training data
cross_validate(model1, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0796  1.1080  1.0635  1.1147  1.1149  1.0962  0.0209  
MAE (testset)     0.8747  0.9034  0.8608  0.9019  0.8980  0.8878  0.0170  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.01    0.01    0.01    0.01    0.00    0.01    0.00    


{'test_rmse': array([1.07961083, 1.10802754, 1.06349711, 1.11472564, 1.1149135 ]),
 'test_mae': array([0.87474507, 0.90339622, 0.86075761, 0.90187401, 0.89800046]),
 'fit_time': (0.002332925796508789,
  0.0029931068420410156,
  0.002651214599609375,
  0.0029129981994628906,
  0.002866983413696289),
 'test_time': (0.005415201187133789,
  0.006418943405151367,
  0.005324840545654297,
  0.0050427913665771484,
  0.004937887191772461)}

In [38]:
#use model to predict
model1.predict(uid=10, iid = 20)
#this model just predicted what user 10 would predict movie 20

Prediction(uid=10, iid=20, r_ui=None, est=3.4550965885737774, details={'was_impossible': True, 'reason': 'Not enough neighbors.'})

looking at the output above we can determine that our model predicted that use 10 would rate movie 20 a 3.5 give or take our RSME which is 1.09.

## Models

In [42]:
model2 = SVD()

In [41]:
model2

{'rmse': <surprise.prediction_algorithms.knns.KNNBasic at 0x7fe4e8b8ef70>,
 'mae': <surprise.prediction_algorithms.knns.KNNBasic at 0x7fe4e8b8eca0>}

In [43]:
cross_validate(model2, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0356  1.0404  0.9768  0.9576  0.9941  1.0009  0.0325  
MAE (testset)     0.8288  0.8243  0.7831  0.7635  0.7899  0.7979  0.0250  
Fit time          0.27    0.26    0.26    0.26    0.26    0.26    0.00    
Test time         0.00    0.00    0.05    0.00    0.00    0.01    0.02    


{'test_rmse': array([1.03560986, 1.04042731, 0.97681723, 0.95758255, 0.99408708]),
 'test_mae': array([0.82876726, 0.82429319, 0.78311272, 0.76346384, 0.78993454]),
 'fit_time': (0.2659456729888916,
  0.26498985290527344,
  0.2602050304412842,
  0.2555968761444092,
  0.2567873001098633),
 'test_time': (0.0049779415130615234,
  0.004007101058959961,
  0.046720027923583984,
  0.0041582584381103516,
  0.004305124282836914)}

In [47]:
param_grid = {'n_factors':[10,20,50],'n_epochs': [5, 10, 15], 'lr_all': [0.002,0.005,0.01],
             'reg_all': [0.2,0.4,0.6]}
gs_model2 = GridSearchCV(SVD,param_grid=param_grid,joblib_verbose=5)
gs_model2.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 405 out of 405 | elapsed:   23.4s finished


In [48]:
gs_model2.best_params

{'rmse': {'n_factors': 10, 'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.2},
 'mae': {'n_factors': 10, 'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.2}}

In [49]:
cross_validate(SVD(n_factors=10,n_epochs=15,lr_all=0.01,reg_all=0.2), train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0041  1.0382  0.9967  0.9921  0.9768  1.0016  0.0204  
MAE (testset)     0.7859  0.8175  0.7950  0.7947  0.7917  0.7970  0.0108  
Fit time          0.05    0.05    0.05    0.05    0.05    0.05    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([1.00413766, 1.03821948, 0.99666895, 0.99206048, 0.97680157]),
 'test_mae': array([0.78588526, 0.81751168, 0.79496702, 0.7947379 , 0.79168374]),
 'fit_time': (0.04806089401245117,
  0.04984593391418457,
  0.04759502410888672,
  0.05060982704162598,
  0.04948568344116211),
 'test_time': (0.004427909851074219,
  0.004129886627197266,
  0.0040912628173828125,
  0.0043790340423583984,
  0.003941059112548828)}

In [52]:
param_grid = {'n_factors':[5,10,15],'n_epochs': [15, 30, 45], 'lr_all': [0.01,0.05,0.1],
             'reg_all': [0.1,0.2,0.3]}
gs_model2 = GridSearchCV(SVD,param_grid=param_grid,joblib_verbose=5)
gs_model2.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 405 out of 405 | elapsed:   40.8s finished


In [53]:
gs_model2.best_params

{'rmse': {'n_factors': 5, 'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.1},
 'mae': {'n_factors': 15, 'n_epochs': 45, 'lr_all': 0.01, 'reg_all': 0.1}}

In [54]:
cross_validate(SVD(n_factors=5,n_epochs=15,lr_all=0.01,reg_all=0.1), train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9714  1.0133  0.9712  1.0320  1.0074  0.9991  0.0241  
MAE (testset)     0.7734  0.8045  0.7791  0.8225  0.7937  0.7947  0.0177  
Fit time          0.04    0.04    0.04    0.04    0.04    0.04    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([0.97138527, 1.01325643, 0.97119057, 1.03200552, 1.00741942]),
 'test_mae': array([0.77337588, 0.80453928, 0.77912972, 0.8225018 , 0.79374782]),
 'fit_time': (0.04303288459777832,
  0.03866291046142578,
  0.0405731201171875,
  0.04176497459411621,
  0.040319204330444336),
 'test_time': (0.0042438507080078125,
  0.003975868225097656,
  0.0047149658203125,
  0.004266023635864258,
  0.004435300827026367)}

In [55]:
model3 = NMF()

In [56]:
cross_validate(model3, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0959  1.0791  1.1005  1.0956  1.1106  1.0964  0.0102  
MAE (testset)     0.8891  0.8799  0.8901  0.8877  0.8923  0.8878  0.0042  
Fit time          0.50    0.50    0.51    0.50    0.50    0.50    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([1.09590589, 1.07908291, 1.10053269, 1.0956239 , 1.110646  ]),
 'test_mae': array([0.88905671, 0.87990915, 0.89008622, 0.887695  , 0.89226315]),
 'fit_time': (0.5038161277770996,
  0.5042572021484375,
  0.5070650577545166,
  0.5023689270019531,
  0.5039169788360596),
 'test_time': (0.004484891891479492,
  0.004911899566650391,
  0.0046160221099853516,
  0.004983186721801758,
  0.004991054534912109)}

In [57]:
param_grid = {'n_factors':[10,15,20],'n_epochs': [25, 50, 75]}
gs_model3 = GridSearchCV(NMF,param_grid=param_grid,joblib_verbose=5)
gs_model3.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   22.4s finished


In [58]:
gs_model3.best_params

{'rmse': {'n_factors': 10, 'n_epochs': 25},
 'mae': {'n_factors': 10, 'n_epochs': 25}}

In [60]:
cross_validate(NMF(n_factors=10,n_epochs=25), train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0993  1.0692  1.1118  1.1191  1.0815  1.0962  0.0185  
MAE (testset)     0.8841  0.8691  0.9036  0.9027  0.8792  0.8877  0.0135  
Fit time          0.20    0.20    0.20    0.20    0.20    0.20    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([1.09932531, 1.06922805, 1.1118063 , 1.11905218, 1.08146487]),
 'test_mae': array([0.88409379, 0.86912547, 0.9036171 , 0.90271361, 0.87919364]),
 'fit_time': (0.20417118072509766,
  0.1975250244140625,
  0.2029118537902832,
  0.19795584678649902,
  0.20138001441955566),
 'test_time': (0.004377126693725586,
  0.004374027252197266,
  0.004478931427001953,
  0.004584074020385742,
  0.004590034484863281)}

In [59]:
param_grid = {'n_factors':[3,5,10],'n_epochs': [15, 20, 25]}
gs_model3 = GridSearchCV(NMF,param_grid=param_grid,joblib_verbose=5)
gs_model3.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    5.7s finished


In [61]:
gs_model3.best_params

{'rmse': {'n_factors': 3, 'n_epochs': 15},
 'mae': {'n_factors': 3, 'n_epochs': 15}}